In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch_numopt
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error

In [2]:
device = "cpu"

In [3]:
class Net(nn.Module):
    def __init__(self, input_size, device="cpu"):
        super().__init__()
        self.f1 = nn.Linear(input_size, 10, device=device)
        self.f2 = nn.Linear(10, 20, device=device)
        self.f3 = nn.Linear(20, 20, device=device)
        self.f4 = nn.Linear(20, 10, device=device)
        self.f5 = nn.Linear(10, 1, device=device)

        self.activation = nn.ReLU()
        # self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.activation(self.f1(x))
        x = self.activation(self.f2(x))
        x = self.activation(self.f3(x))
        x = self.activation(self.f4(x))
        x = self.f5(x)

        return x

In [4]:
# X, y = load_diabetes(return_X_y=True, scaled=False)
# X, y = make_regression(n_samples=1000, n_features=100)
X, y = make_friedman1(n_samples=1000, noise=1e-2)

X_scaler = MinMaxScaler()
X = X_scaler.fit_transform(X)
X = torch.Tensor(X).to(device)

y_scaler = MinMaxScaler()
y = y_scaler.fit_transform(y.reshape((-1, 1)))
y = torch.Tensor(y).to(device)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

torch_data = TensorDataset(X_train, y_train)
data_loader = DataLoader(torch_data, batch_size=1000)

torch.Size([800, 10]) torch.Size([800, 1])
torch.Size([200, 10]) torch.Size([200, 1])


In [7]:
model = Net(input_size=X.shape[1], device=device)
loss_fn = nn.MSELoss()

# opt = torch_numopt.ConjugateGradient(model=model, lr=1e-4, line_search_method="const", cg_method="FR")
opt = torch_numopt.ConjugateGradient(model=model, lr=10, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="armijo", cg_method="DY")
# opt = torch_numopt.ConjugateGradient(model=model, lr=10, c1=0.1, c2=0.9, tau=0.1, line_search_method="backtrack", line_search_cond="wolfe", cg_method="FR")
# opt = torch_numopt.ConjugateGradient(model=model, lr=10, c1=0.1, c2=0.9, tau=0.1, line_search_method="backtrack", line_search_cond="strong-wolfe", cg_method="FR")
# opt = torch_numopt.ConjugateGradient(model=model, lr=10, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="goldstein", cg_method="FR")

all_loss = {}
for epoch in range(1000):
    print("epoch: ", epoch, end="")
    all_loss[epoch + 1] = 0
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)

        all_loss[epoch + 1] += loss
    all_loss[epoch + 1] /= len(data_loader)
    print(", loss: {}".format(all_loss[epoch + 1].detach().numpy().item()))

epoch:  0, loss: 0.40786683559417725
epoch:  1, loss: 0.23815593123435974
epoch:  2, loss: 0.23815593123435974
epoch:  3, loss: 0.04747726395726204
epoch:  4, loss: 0.04747726395726204
epoch:  5, loss: 0.03552425280213356
epoch:  6, loss: 0.03432055562734604
epoch:  7, loss: 0.03406721353530884
epoch:  8, loss: 0.03406721353530884
epoch:  9, loss: 0.03391239419579506
epoch:  10, loss: 0.033842816948890686
epoch:  11, loss: 0.033842816948890686
epoch:  12, loss: 0.033614709973335266
epoch:  13, loss: 0.03319547325372696
epoch:  14, loss: 0.03319384902715683
epoch:  15, loss: 0.032414402812719345
epoch:  16, loss: 0.032264236360788345
epoch:  17, loss: 0.032042406499385834
epoch:  18, loss: 0.03204220533370972
epoch:  19, loss: 0.03195972740650177
epoch:  20, loss: 0.031941007822752
epoch:  21, loss: 0.031478844583034515
epoch:  22, loss: 0.031478844583034515
epoch:  23, loss: 0.03147903457283974
epoch:  24, loss: 0.03143925964832306
epoch:  25, loss: 0.03090745396912098
epoch:  26, loss

In [8]:
pred_train = model.forward(X_train).detach()
pred_test = model.forward(X_test).detach()
print(f"Train metrics: R2 = {r2_score(pred_train, y_train)}")
print(f"Test metrics:  R2 = {r2_score(pred_test, y_test)}")

Train metrics: R2 = 0.8762005858713022
Test metrics:  R2 = 0.8832225145901158


In [7]:
model = Net(input_size=X.shape[1], device=device)
loss_fn = nn.MSELoss()

# opt = torch_numopt.ConjugateGradient(model=model, lr=1e-4, line_search_method="const", cg_method="PR")
opt = torch_numopt.ConjugateGradient(model=model, lr=10, c1=0.01, tau=0.1, line_search_method="backtrack", line_search_cond="armijo", cg_method="PR")
# opt = torch_numopt.ConjugateGradient(model=model, lr=10, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="wolfe", cg_method="PR")
# opt = torch_numopt.ConjugateGradient(model=model, lr=10, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="strong-wolfe", cg_method="PR")
# opt = torch_numopt.ConjugateGradient(model=model, lr=10, c1=1e-6, tau=0.1, line_search_method="backtrack", line_search_cond="goldstein", cg_method="PR")


all_loss = {}
for epoch in range(1000):
    print("epoch: ", epoch, end="")
    all_loss[epoch + 1] = 0
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)

        all_loss[epoch + 1] += loss
    all_loss[epoch + 1] /= len(data_loader)
    print(", loss: {}".format(all_loss[epoch + 1].detach().numpy().item()))

epoch:  0, loss: 0.4871537685394287
epoch:  1, loss: 0.2800534665584564
epoch:  2, loss: 0.14550399780273438
epoch:  3, loss: 0.0787799134850502
epoch:  4, loss: 0.05023331567645073
epoch:  5, loss: 0.03950706124305725
epoch:  6, loss: 0.035870056599378586
epoch:  7, loss: 0.03470418602228165
epoch:  8, loss: 0.03432386741042137
epoch:  9, loss: 0.03418225049972534
epoch:  10, loss: 0.034110717475414276
epoch:  11, loss: 0.033823247998952866
epoch:  12, loss: 0.033823247998952866
epoch:  13, loss: 0.0337141789495945
epoch:  14, loss: 0.03364672511816025
epoch:  15, loss: 0.03348459303379059
epoch:  16, loss: 0.03348459303379059
epoch:  17, loss: 0.03333776071667671
epoch:  18, loss: 0.03325455263257027
epoch:  19, loss: 0.03315816819667816
epoch:  20, loss: 0.03315816819667816
epoch:  21, loss: 0.03295530378818512
epoch:  22, loss: 0.0328504778444767
epoch:  23, loss: 0.03284872695803642
epoch:  24, loss: 0.03284872695803642
epoch:  25, loss: 0.0325714647769928
epoch:  26, loss: 0.0324

In [8]:
pred_train = model.forward(X_train).detach()
pred_test = model.forward(X_test).detach()
print(f"Train metrics: R2 = {r2_score(pred_train, y_train)}")
print(f"Test metrics:  R2 = {r2_score(pred_test, y_test)}")

Train metrics: R2 = 0.8501156290473114
Test metrics:  R2 = 0.7606448171390976


In [9]:
model = Net(input_size=X.shape[1], device=device)
loss_fn = nn.MSELoss()

# opt = torch_numopt.ConjugateGradient(model=model, lr=1e-4, line_search_method="const", cg_method="PR")
opt = torch_numopt.ConjugateGradient(model=model, lr=10, c1=0.01, tau=0.1, line_search_method="backtrack", line_search_cond="armijo", cg_method="PRP+")
# opt = torch_numopt.ConjugateGradient(model=model, lr=10, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="wolfe", cg_method="PRP+")
# opt = torch_numopt.ConjugateGradient(model=model, lr=10, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="strong-wolfe", cg_method="PRP+")
# opt = torch_numopt.ConjugateGradient(model=model, lr=10, c1=1e-6, tau=0.1, line_search_method="backtrack", line_search_cond="goldstein", cg_method="PRP")


all_loss = {}
for epoch in range(1000):
    print("epoch: ", epoch, end="")
    all_loss[epoch + 1] = 0
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)

        all_loss[epoch + 1] += loss
    all_loss[epoch + 1] /= len(data_loader)
    print(", loss: {}".format(all_loss[epoch + 1].detach().numpy().item()))

epoch:  0, loss: 0.3715151250362396
epoch:  1, loss: 0.22022412717342377
epoch:  2, loss: 0.1412205547094345
epoch:  3, loss: 0.09496459364891052
epoch:  4, loss: 0.06842473149299622
epoch:  5, loss: 0.053537026047706604
epoch:  6, loss: 0.04536965861916542
epoch:  7, loss: 0.040975283831357956
epoch:  8, loss: 0.038647860288619995
epoch:  9, loss: 0.03742722421884537
epoch:  10, loss: 0.036788277328014374
epoch:  11, loss: 0.03644932806491852
epoch:  12, loss: 0.03626001253724098
epoch:  13, loss: 0.03624358028173447
epoch:  14, loss: 0.03624358028173447
epoch:  15, loss: 0.035934075713157654
epoch:  16, loss: 0.03578561916947365
epoch:  17, loss: 0.03570716455578804
epoch:  18, loss: 0.03566036373376846
epoch:  19, loss: 0.035545866936445236
epoch:  20, loss: 0.035545866936445236
epoch:  21, loss: 0.035459887236356735
epoch:  22, loss: 0.03540756180882454
epoch:  23, loss: 0.03528168797492981
epoch:  24, loss: 0.03528168797492981
epoch:  25, loss: 0.03518339991569519
epoch:  26, loss

In [10]:
pred_train = model.forward(X_train).detach()
pred_test = model.forward(X_test).detach()
print(f"Train metrics: R2 = {r2_score(pred_train, y_train)}")
print(f"Test metrics:  R2 = {r2_score(pred_test, y_test)}")

Train metrics: R2 = 0.782798339195211
Test metrics:  R2 = 0.6721940797246311
